In [1]:
from PEMD.analysis import msd as msd_module, conductivity, transfer_number

### 1. Prepare the input parameter 

In [2]:
work_dir = '/home/tsd/polymer/MD/PEO/LI_EO/0.3/375/3_sample/MD_dir'

data_tpr_file = 'nvt_prod.tpr'
dcd_xtc_file = 'unwrapped_traj.xtc'
select_cations = 'resname LIP'  
select_anions = 'resname NSC'

dt = 0.001
dt_collection = 5e3
run_start = int(1e6)
nsteps = 4e8
T = 375 # K
interval_time = 5e3     # slope range 5ns

### 2. Obtain the atom position

In [3]:
# Calculate the positions of ions in a molecular dynamics simulation.

(
    run, 
    cations, 
    cations_list, 
    anions, 
    anions_list, 
    times,
) = msd_module.get_position(
    work_dir, 
    data_tpr_file, 
    dcd_xtc_file, 
    select_cations, 
    select_anions, 
    dt, 
    dt_collection, 
    run_start, 
    nsteps,
    format='GROMACS',
)

In [4]:
# Generates arrays of position data for cations and anions relative to the center of mass of the system.

(
    cation_positions, 
    anion_positions,
)= msd_module.create_position_arrays(
    run, 
    cations_list, 
    anions_list, 
    times, 
    run_start, 
    dt_collection,
)

  0%|          | 0/79801 [00:00<?, ?it/s]

### 3. Calculate the MSD and Self-Diffusion Coefficient

In [ ]:
# Calculate the self-diffusion coefficient for cation.
(
    msd_cation, 
    D_cation, 
    time_range_cation,
) = msd_module.compute_self_diffusion(
    cation_positions, 
    times, 
    dt_collection, 
    dt, 
    interval_time,
)

# Calculate the self-diffusion coefficient for anion.
(
    msd_anion, 
    D_anion, 
    time_range_anion,
) = msd_module.compute_self_diffusion(
    anion_positions, 
    times, 
    dt_collection, 
    dt, 
    interval_time,
)

Calculating MSD:   0%|          | 0/300 [00:00<?, ?it/s]

Calculating MSD:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
D_cation

In [ ]:
D_anion

In [ ]:
# plotting log-log scale mean squared displacement (MSD) for Ion.
import matplotlib.pyplot as plt

font_list = {"title" : 20, "label":18, "legend":16, "ticket": 18, "data": 14} 
linewith = 1.5
markersize = 5
color_list = ["#DF543F", "#2286A9", "#FBBF7C", "#3C3846"]

x_log = times[2000:6000]
y_log = x_log/80
x_log2 = times[50000:10000000000]
y_log2 = x_log2/700
fig, ax = plt.subplots()
ax.plot(times[1:], msd_cation[1:], '-', linewidth=linewith, color = color_list[0], label="Li$^+$")
ax.plot(times[1:], msd_anion[1:], '-', linewidth=linewith, color = color_list[1], label="TFSI$^-$")

ax.plot(x_log, y_log, '--', linewidth=2, color="grey")
ax.plot(x_log2, y_log2, '--', linewidth=2, color="grey")

ax.legend(fontsize=font_list["legend"], frameon=False)
ax.set_xlabel(r'$t$ (ps)', fontsize=font_list["label"])
ax.set_ylabel(r'MSD ($\AA^2$)', fontsize=font_list["label"])
ax.tick_params(axis='both', which='both', direction='in',labelsize=font_list["ticket"])
# ax.tick_params(axis='y', labelsize=font_list["ticket"], direction='in', length=6, width=2)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim()
ax.set_xlim(1e2,)
# ax.set_ylim(1e0, 2e2)

ax.grid(True, linestyle='--')
fig.set_size_inches(5.5,4)
plt.tight_layout()

# Save the plot
plt.savefig('msd.tif', bbox_inches='tight', dpi=300)
plt.show()

### 4. Calculate the conductivity

In [ ]:
(
    msd_total, 
    cond, 
    time_range
) = conductivity.compute_conductivity(
    run, 
    run_start, 
    dt_collection, 
    cations_list, 
    anions_list, 
    times, 
    dt, 
    T,
    interval_time
)

In [ ]:
cond

In [ ]:
# plotting log-log scale mean squared displacement (MSD) for Ion.
import matplotlib.pyplot as plt

font_list = {"title" : 20, "label":18, "legend":16, "ticket": 18, "data": 14} 
linewith = 1.5
markersize = 5
color_list = ["#DF543F", "#2286A9", "#FBBF7C", "#3C3846"]

#time = np.arange(run_start/100, run_end/100, 1/100)
# time = np.arange(run_start*10, run_end*10, 10)   # timestep:10 ps, unit: ps
x_log = times[800:3000]
y_log = x_log/1
# x_log2 = times[18000:40000]
# y_log2 = x_log2/2800
fig, ax = plt.subplots()
ax.plot(times[1:], msd_total[1:], '-', linewidth=linewith, color = color_list[0], label="PEO/LiTFSI")
# ax.plot(times[1:], msd_anion[1:], '-', linewidth=linewith, color = color_list[1], label="TFSI$^-$")

ax.plot(x_log, y_log, '--', linewidth=2, color="grey")
# ax.plot(x_log2, y_log2, '--', linewidth=2, color="grey")

ax.legend(fontsize=font_list["legend"], frameon=False)
ax.set_xlabel(r'$t$ (ps)', fontsize=font_list["label"])
ax.set_ylabel(r'MSD ($\AA^2$)', fontsize=font_list["label"])
ax.tick_params(axis='both', which='both', direction='in',labelsize=font_list["ticket"])
# ax.tick_params(axis='y', labelsize=font_list["ticket"], direction='in', length=6, width=2)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(100, 200000)
# ax.set_ylim(1e0, 2e2)

ax.grid(True, linestyle='--')
fig.set_size_inches(5.5,4)
plt.tight_layout()

# Save the plot
plt.savefig('msd.tif', bbox_inches='tight', dpi=300)
plt.show()

### 5. calculate the transfer number

In [ ]:
t = transfer_number.compute_transfer_number(
    run, 
    dt_collection, 
    cation_positions, 
    anion_positions, 
    times, 
    dt, 
    T, 
    interval_time, 
    cond
)

In [ ]:
t